## Testing Cuda

In [1]:
# Now verify our GPU setup
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("GPU device name:", torch.cuda.get_device_name(0))
    print("GPU memory allocated:", torch.cuda.memory_allocated(0)/1024**2, "MB")
    print("GPU memory reserved:", torch.cuda.memory_reserved(0)/1024**2, "MB")

PyTorch version: 2.6.0+cu126
CUDA available: True
CUDA version: 12.6
GPU device name: NVIDIA GeForce RTX 3060
GPU memory allocated: 0.0 MB
GPU memory reserved: 0.0 MB


## Load Dataset

In [2]:
import pandas as pd
from datasets import Dataset, DatasetDict
import os

def load_urdu_dataset(data_dir):
    """
    Load Urdu dataset from local TSV files
    data_dir: Directory containing train.tsv, dev.tsv, test.tsv and clips folder
    """
    # Load TSV files
    train_df = pd.read_csv(os.path.join(data_dir, 'train.tsv'), sep='\t')
    dev_df = pd.read_csv(os.path.join(data_dir, 'dev.tsv'), sep='\t')
    test_df = pd.read_csv(os.path.join(data_dir, 'test.tsv'), sep='\t')
    
    # Add full audio path
    clips_dir = os.path.join(data_dir, 'clips')
    train_df['audio'] = train_df['path'].apply(lambda x: os.path.join(clips_dir, x))
    dev_df['audio'] = dev_df['path'].apply(lambda x: os.path.join(clips_dir, x))
    test_df['audio'] = test_df['path'].apply(lambda x: os.path.join(clips_dir, x))
    
    # Keep only necessary columns
    columns_to_keep = ['audio', 'sentence']
    train_df = train_df[columns_to_keep]
    dev_df = dev_df[columns_to_keep]
    test_df = test_df[columns_to_keep]
    
    # Convert to Hugging Face datasets
    train_dataset = Dataset.from_pandas(train_df)
    dev_dataset = Dataset.from_pandas(dev_df)
    test_dataset = Dataset.from_pandas(test_df)
    
    # Combine into DatasetDict
    dataset_dict = DatasetDict({
        'train': train_dataset,
        'validation': dev_dataset,
        'test': test_dataset
    })
    
    return dataset_dict

# Load the dataset
# Replace with your actual data directory path
data_dir = "S:/cv-corpus-20.0-2024-12-06/ur"
urdu_dataset = load_urdu_dataset(data_dir)

# Print dataset statistics
print("\nDataset statistics:")
for split in urdu_dataset:
    print(f"{split}: {len(urdu_dataset[split])} examples")

# Print first example from training set
print("\nExample from training set:")
print(urdu_dataset['train'][0])


Dataset statistics:
train: 7205 examples
validation: 4982 examples
test: 5026 examples

Example from training set:
{'audio': 'S:/cv-corpus-20.0-2024-12-06/ur\\clips\\common_voice_ur_31898340.mp3', 'sentence': 'طرح طرح کے پرندے'}


In [3]:
urdu_dataset

DatasetDict({
    train: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 7205
    })
    validation: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 4982
    })
    test: Dataset({
        features: ['audio', 'sentence'],
        num_rows: 5026
    })
})

## Prepare Feature Extractor, Tokenizer and Data

In [4]:
from transformers import WhisperFeatureExtractor, WhisperTokenizer, WhisperProcessor

# Initialize the feature extractor
feature_extractor = WhisperFeatureExtractor.from_pretrained("openai/whisper-tiny")

# Initialize the tokenizer with Urdu language
tokenizer = WhisperTokenizer.from_pretrained("openai/whisper-tiny", language="Urdu", task="transcribe")

# Let's also create the processor which combines both feature_extractor and tokenizer
processor = WhisperProcessor.from_pretrained("openai/whisper-tiny", language="Urdu", task="transcribe")

# Let's test the tokenizer with a sample Urdu text from our dataset
sample_text = urdu_dataset["train"][0]["sentence"]
print("\nTesting tokenizer:")
print("Original text:", sample_text)
encoded = tokenizer(sample_text)
print("Encoded text:", encoded)

decoded = tokenizer.decode(encoded.input_ids)
print("Decoded text:", decoded)

# Let's also look at the feature extractor's configuration
print("\nFeature extractor config:")
print("Sampling rate:", feature_extractor.sampling_rate)
print("Feature size:", feature_extractor.feature_size)
print("Padding value:", feature_extractor.padding_value)


Testing tokenizer:
Original text: طرح طرح کے پرندے
Encoded text: {'input_ids': [50258, 50290, 50359, 50363, 9566, 2288, 5016, 23032, 2288, 5016, 24049, 21453, 2288, 41260, 7369, 50257], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Decoded text: <|startoftranscript|><|ur|><|transcribe|><|notimestamps|>طرح طرح کے پرندے<|endoftext|>

Feature extractor config:
Sampling rate: 16000
Feature size: 80
Padding value: 0.0


## Testing tokenizer

In [5]:
# Test tokenizer with and without special tokens
input_str = urdu_dataset["train"][0]["sentence"]
labels = tokenizer(input_str).input_ids
decoded_with_special = tokenizer.decode(labels, skip_special_tokens=False)
decoded_str = tokenizer.decode(labels, skip_special_tokens=True)

print("\nTokenizer test with special tokens:")
print(f"Input:                 {input_str}")
print(f"Decoded w/ special:    {decoded_with_special}")
print(f"Decoded w/out special: {decoded_str}")
print(f"Are equal:             {input_str == decoded_str}")


Tokenizer test with special tokens:
Input:                 طرح طرح کے پرندے
Decoded w/ special:    <|startoftranscript|><|ur|><|transcribe|><|notimestamps|>طرح طرح کے پرندے<|endoftext|>
Decoded w/out special: طرح طرح کے پرندے
Are equal:             True


## check default sampling rate of 1st training example

In [6]:
print(urdu_dataset["train"][0])

{'audio': 'S:/cv-corpus-20.0-2024-12-06/ur\\clips\\common_voice_ur_31898340.mp3', 'sentence': 'طرح طرح کے پرندے'}


## fixing sampling rate

In [7]:
# Now let's prepare our dataset with the correct sampling rate
# First, let's check the current audio format
from datasets import Audio
import numpy as np

# Cast the audio column to audio format
urdu_dataset = urdu_dataset.cast_column("audio", Audio(sampling_rate=16000))

# Let's examine the first audio file after casting
print("\nAudio format check:")
print("First example audio details:")
audio = urdu_dataset["train"][0]["audio"]
print(f"Path: {audio['path']}")
print(f"Sampling rate: {audio['sampling_rate']}")
print(f"Array shape: {audio['array'].shape}")


Audio format check:
First example audio details:
Path: S:/cv-corpus-20.0-2024-12-06/ur\clips\common_voice_ur_31898340.mp3
Sampling rate: 16000
Array shape: (50688,)


In [8]:
#check the format of 1st training example
print(urdu_dataset["train"][0])


{'audio': {'path': 'S:/cv-corpus-20.0-2024-12-06/ur\\clips\\common_voice_ur_31898340.mp3', 'array': array([1.39698386e-09, 1.74622983e-09, 2.91038305e-10, ...,
       1.17505246e-04, 1.56884955e-04, 4.67942300e-05]), 'sampling_rate': 16000}, 'sentence': 'طرح طرح کے پرندے'}


In [9]:
def prepare_dataset(batch, processor):
    """
    Prepare the dataset for training, leaving padding and tensor conversion to the data collator:
    1. Load and resample audio
    2. Compute log-Mel input features
    3. Encode target text to label ids (without padding or tensor conversion)
    """
    # Load audio
    audio = batch["audio"]
    
    # Compute log-Mel input features
    batch["input_features"] = processor.feature_extractor(
        audio["array"], 
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]
    
    # Encode target text to label ids (without padding)
    batch["labels"] = processor.tokenizer(
        batch["sentence"],
        max_length=1024,
        truncation=True
    ).input_ids

    return batch

In [10]:
# Process dataset split by split with very small batches
def process_and_save_split(dataset_split, split_name, processor, save_dir="./processed_dataset"):
    """Process a single split of the dataset with minimal memory usage"""
    os.makedirs(save_dir, exist_ok=True)
    
    # Process in chunks of 100 examples
    chunk_size = 100
    processed_chunks = []
    
    for i in range(0, len(dataset_split), chunk_size):
        end_idx = min(i + chunk_size, len(dataset_split))
        print(f"\nProcessing {split_name} examples {i} to {end_idx}")
        
        # Select a small chunk
        chunk = dataset_split.select(range(i, end_idx))
        
        # Process the chunk
        processed_chunk = chunk.map(
            function=prepare_dataset,
            fn_kwargs={"processor": processor},
            remove_columns=chunk.column_names,
            desc=f"Processing chunk {i}-{end_idx}",
            batch_size=4,  # Very small batch size
            writer_batch_size=50  # Small writer batch size
        )
        
        # Save the processed chunk immediately
        chunk_path = f"{save_dir}/{split_name}_chunk_{i}"
        processed_chunk.save_to_disk(chunk_path)
        processed_chunks.append(chunk_path)
        
        # Clear memory
        del chunk, processed_chunk
        import gc
        gc.collect()
    
    # Combine all chunks for this split
    print(f"\nCombining chunks for {split_name} split...")
    combined_split = concatenate_datasets([load_from_disk(path) for path in processed_chunks])
    
    # Save the combined split
    combined_split.save_to_disk(f"{save_dir}/{split_name}")
    
    # Clean up chunk files
    for chunk_path in processed_chunks:
        import shutil
        shutil.rmtree(chunk_path)
    
    return combined_split

# Process each split separately
try:
    from datasets import concatenate_datasets, load_from_disk
    
    print("Processing dataset split by split...")
    processed_dataset = {}
    
    for split in urdu_dataset.keys():
        print(f"\nProcessing {split} split...")
        processed_dataset[split] = process_and_save_split(
            urdu_dataset[split], 
            split, 
            processor,
            save_dir="./processed_urdu_dataset"
        )
    
    # Create the final DatasetDict
    from datasets import DatasetDict
    urdu_dataset = DatasetDict(processed_dataset)
    
    print("\nDataset processing completed!")
    
    # Verify the processed dataset
    for split in urdu_dataset.keys():
        print(f"\n{split} split size:", len(urdu_dataset[split]))
        
except Exception as e:
    print(f"An error occurred: {str(e)}")
    raise e

Processing dataset split by split...

Processing train split...

Processing train examples 0 to 100


Processing chunk 0-100:   0%|          | 0/100 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/100 [00:00<?, ? examples/s]


Processing train examples 100 to 200


Processing chunk 100-200:   0%|          | 0/100 [00:00<?, ? examples/s]

...

Processing chunk 5000-5026:   0%|          | 0/26 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/26 [00:00<?, ? examples/s]


Combining chunks for test split...


Saving the dataset (0/10 shards):   0%|          | 0/5026 [00:00<?, ? examples/s]


Dataset processing completed!

train split size: 7205

validation split size: 4982

test split size: 5026


## Load Preprocessed dataset From Disk

In [13]:
from datasets import load_from_disk, DatasetDict

# Load each split separately
urdu_dataset = DatasetDict({
    "train": load_from_disk("./processed_urdu_dataset/train"),
    "validation": load_from_disk("./processed_urdu_dataset/validation"),
    "test": load_from_disk("./processed_urdu_dataset/test")
})

# Check the first example from each split
for split in urdu_dataset.keys():
    print(f"\n=== Example from {split} split ===")
    example = urdu_dataset[split][0]
    
    # Print the input features shape
    print(f"Input features shape: {example['input_features'].shape}")
    
    # Decode the labels back to text
    decoded_text = processor.tokenizer.decode(example['labels'], skip_special_tokens=True)
    print(f"Transcription: {decoded_text}")
    
# Print dataset sizes
print("\n=== Dataset Statistics ===")
for split in urdu_dataset.keys():
    print(f"{split} split size: {len(urdu_dataset[split])} examples")


=== Example from train split ===


AttributeError: 'list' object has no attribute 'shape'

In [12]:

# Print dataset format
print("\nDataset format:")
print(urdu_dataset)


Dataset format:
DatasetDict({
    train: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 7205
    })
    validation: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 4982
    })
    test: Dataset({
        features: ['input_features', 'labels'],
        num_rows: 5026
    })
})


In [18]:
from huggingface_hub import login
# Login to Hugging Face (you'll be prompted for your token)
login()

In [21]:
# # Push the dataset to the Hub
# dataset_name = "urdu-common-voice-20-processed-large-v3"
# urdu_dataset.push_to_hub(
#     f"osman31/{dataset_name}",
#     private=True  # Set to False if you want it public
# )

# print(f"Dataset pushed to: https://huggingface.co/datasets/osman31/{dataset_name}")

Uploading the dataset shards:   0%|          | 0/23 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Dataset pushed to: https://huggingface.co/datasets/osman31/urdu-common-voice-20-processed-large-v3


# Step 2 - Training